# SQL BigQuery

O desafio aqui será responder duas perguntas a partir do Dataset fornecido e usando apenas SQL.


In [1]:
%load_ext google.cloud.bigquery

Para testar o uso da API e das credenciais do Google, executo a query já existente no desafio, que lista 
os 5 produtos que mais geraram receita. Com a consulta retornando "ok", agora passaremos as duas questões do desafio.

In [3]:
%%bigquery

SELECT
 properties.product,
 
SUM(properties.revenue) revenue
FROM `dito-data-scientist-challenge.tracking.dito`
WHERE type = 'track'
GROUP BY properties.product
ORDER BY revenue DESC
LIMIT 5

,product,revenue
0,Schneider Aventinus,52878.34
1,"Racer 5 India Pale Ale, Bear Republic Bre",52034.75
2,Old Rasputin Russian Imperial Stout,51862.79
3,Trappistes Rochefort 10,51216.23
4,Stone IPA,50754.15


### 1. Qual o nome, email e telefone das 5 pessoas que mais geraram receita?

Para esse exercício, primeiro busquei pelos id's das pessoas que mais geraram receita, salvando os resultados dos id's em um dataframe.

In [4]:
from google.cloud import bigquery
from pandas import DataFrame

client = bigquery.Client()

sql = """
SELECT
    id,
    SUM(properties.revenue) revenue
FROM `dito-data-scientist-challenge.tracking.dito`
GROUP BY id
ORDER BY SUM(properties.revenue) DESC
LIMIT 5
"""

df = client.query(sql).to_dataframe()
df.head()

,id,revenue
0,f56861d3-a2b5-4ab0-b0ef-6ce2005f84dd,4850.27
1,bea16033-7519-4520-a2ba-0425174d397b,4433.44
2,9e505eb0-e3f2-4715-a1e9-e05605022a95,4289.69
3,99ab1cee-2645-4ae3-8ffe-f28508666098,4261.43
4,fc3650a4-3f19-4047-9354-65e2a23a4a40,4260.19


In [5]:
ids = df['id'].astype(str).values.tolist()
ids

['f56861d3-a2b5-4ab0-b0ef-6ce2005f84dd',
 'bea16033-7519-4520-a2ba-0425174d397b',
 '9e505eb0-e3f2-4715-a1e9-e05605022a95',
 '99ab1cee-2645-4ae3-8ffe-f28508666098',
 'fc3650a4-3f19-4047-9354-65e2a23a4a40']

Na sequência, posso agora consultar o nome, email e telefone de cada um para exibir sua ordenação, passando a lista de ids retornados na primeira consulta.

In [6]:
client = bigquery.Client()

sql_names = """
SELECT traits.name, traits.email, traits.phone
FROM `dito-data-scientist-challenge.tracking.dito`
WHERE type = 'identify' AND id IN UNNEST(%s)
ORDER BY timestamp DESC
LIMIT 5
""" %(map(str, ids))

names = client.query(sql_names).to_dataframe

In [8]:
names.head()

,nome,email,telefone
0,Heloísa Ordonhes,joshua_prosacco@gmail.com,(51) 91972-9639
1,Mirella Soares,velia.pagac@hotmail.com,(43) 97290-9288
2,Fernanda Resende,charlott.ledner@live.com,(33) 92311-0450
3,Sophie Banheira,fransisca.predovic@bol.com.br,(24) 91404-3805
4,Alícia Pinheira,bill@bol.com.br,(51) 91012-4779


### 2. De quantos em quantos dias, em média, as pessoas compram? Use a mediana como média.


Primeiramente é necessário buscar a data da primeira e da última compra, para então descobrir o número de dias entre ambas. Após isso, podemos usar o resultado dessa seleção para buscar o valor da mediana.

In [9]:
%%bigquery

WITH buy_interval AS(
SELECT  id, DATE_DIFF(MAX(DATE(timestamp)), MIN(DATE(timestamp)), DAY)/COUNT(id) AS freq_mean
FROM `dito-data-scientist-challenge.tracking.dito`
WHERE type='track'
GROUP BY id)

SELECT APPROX_QUANTILES(freq_mean,1000)[OFFSET(500)] mediana
FROM buy_interval
LIMIT 1

,mediana
0,1.038462


Portanto, a pessoas compram, em média, a cada 1 dia.